In [1]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
import random
import numpy as np
import torchvision.utils as vutils


In [2]:
# import my new datasetloader here
# from newX1000loader import X4K1000FPSDataset
from VimeoSeptupletLoader import VimeoDataset as mydataset

In [3]:
seed = 12345 #56789 #1234
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = True

In [4]:

# Custom Dataset Class
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_groups = []

        for subdir in os.listdir(root_dir):
            subdir_path = os.path.join(root_dir, subdir)
            if os.path.isdir(subdir_path):
                images = sorted(os.listdir(subdir_path))
                for i in range(0, len(images) - 2):
                    self.image_groups.append([os.path.join(subdir_path, images[i]),
                                              os.path.join(subdir_path, images[i+1]),
                                              os.path.join(subdir_path, images[i+2])])

    def __len__(self):
        return len(self.image_groups)

    def __getitem__(self, idx):
        image_paths = self.image_groups[idx]
        images = [Image.open(img_path).convert('RGB') for img_path in image_paths]

        if self.transform:
            images = [self.transform(image) for image in images]

        image_stack = torch.stack(images, dim=0)
        return image_stack

# Transformation
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])


In [5]:

# Dataset and DataLoader
transform = transforms.Compose([
    transforms.ToTensor()  # Convert images to tensor after all other transformations
])
# dataset = X4K1000FPSDataset(root_dir='/home/jyzhao/Code/Datasets/X4K1000FPS', transform=transform)
# Vimeo90K 
dataset = mydataset('train')


In [7]:
# TensorBoard Setup
writer = SummaryWriter('runs/dataset_visualization')
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)
print("---start---")
for epoch in range(1):
        #sampler.set_epoch(epoch)
        print("Epoch",epoch)
        for i, data in enumerate(dataloader):
               print('i',i)


        # Log images to TensorBoard
        # Note: Adjust 'make_grid' as needed based on your dataset's image dimensions and properties



---start---
Epoch 0
i 0
i 1
i 2
i 3
i 4


In [11]:
# TensorBoard Setup
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

writer = SummaryWriter('runs/dataset_visualization')

# Fetch a batch of data
images = next(iter(dataloader))[0]  # Get the first batch and extract the image stack
images_to_display = images[0, :, :, :, :]  # This selects all images from the first batch item

# Since make_grid expects a (B, C, H, W) tensor, we treat the 'N' dimension as the batch dimension
grid = vutils.make_grid(images_to_display)
writer.add_image('sample_images_from_first_item', grid, global_step=epoch)

writer.close()
print('Images logged to TensorBoard.')


Images logged to TensorBoard.
